In [1]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
import re
import random

## 数据准备

In [2]:
jl = []

In [3]:
with open('github_python0.jl', 'r') as f:
    for i in f:
        jl.append(json.loads(i))


In [4]:
random.shuffle(jl)

In [5]:
def str2int(x: str):
    if 'k' in x:
        return int(float(x[:-1])*1000)
    else:
        return int(x)


首先对数据进行处理，我选取了`fork`, `release`, `contibutors`, `pulls_open`, `pulls_closed`, `issues_open`, `issues_closed`这几个指标作为输入数据， 用`star`作为标签数据，训练模型。


In [6]:
def process(x):
    star = str2int(x['star'])
    fork = str2int(x['fork'])
    release = int(x['release'])
    contr = int(x['contributors'])
    branch = int(x['branch'])
    topic_len = len(x['topic'])
    lang_len = len(x['languages'])
    pull_open = x['pull_requests_open']
    m = re.match('(^[0-9,]+)', pull_open)
    pull_open = int(m.group(1).replace(',', ''))
    pull_clos = x['pull_requests_closed']
    m = re.match('(^[0-9,]+)', pull_clos)
    pull_clos = int(m.group(1).replace(',', ''))
    issu_open = x['issues_open']
    m = re.match('(^[0-9,]+)', issu_open)
    issu_open = int(m.group(1).replace(',', ''))
    issu_clos = x['issues_closed']
    m = re.match('(^[0-9,]+)', issu_clos)
    issu_clos = int(m.group(1).replace(',', ''))
    data = np.array([fork/1000,
                     release/10, contr,
                     pull_open/100, pull_clos/100,
                     issu_open/100, issu_clos/100]).astype(np.float32)
    lab = np.array([star]).astype(np.float32)
    return data, lab


In [7]:
process(jl[0])

(array([ 1.5 ,  1.4 , 14.  ,  0.18,  8.28,  3.12, 13.52], dtype=float32),
 array([22600.], dtype=float32))

In [8]:
ds = []
for i in jl:
    try:
        ds.append(process(i))
    except:
        print(i['url'])


https://github.com/h2oai/h2o-3
https://github.com/apache/flink
https://github.com/apache/avro
https://github.com/DonJayamanne/pythonVSCode
https://github.com/pytorch/pytorch
https://github.com/itdevbooks/pdf
https://github.com/tianocore/edk2
https://github.com/life4/textdistance
https://github.com/navdeep-G/samplemod
https://github.com/Azure/azure-sdk-for-python
https://github.com/ab77/netflix-proxy
https://github.com/d2l-ai/d2l-zh
https://github.com/justjavac/free-programming-books-zh_CN
https://github.com/sivel/speedtest-cli
https://github.com/Avik-Jain/100-Days-Of-ML-Code
https://github.com/timofurrer/awesome-asyncio
https://github.com/byt3bl33d3r/MITMf
https://github.com/apache/spark
https://github.com/SerpentAI/SerpentAI
https://github.com/mementum/backtrader
https://github.com/fastai/fastpages
https://github.com/dcos/dcos
https://github.com/twisted/twisted
https://github.com/apache/beam
https://github.com/souyunku/DevBooks
https://github.com/apache/activemq
https://github.com/mic

之后将数据处理成符合输入的形式，并切成训练集和测试集。

In [9]:
ds_len = len(ds)
ds_len


990

In [10]:
sp = (ds_len//10)*8
sp

792

In [11]:
train_ds = ds[:sp]
len(train_ds)

792

In [12]:
train_X = []
train_y = []
for x, y in train_ds:
    train_X.append(x)
    train_y.append(y)

train_X = np.array(train_X)
print(train_X.shape)
train_y = np.array(train_y).squeeze()
print(train_y.shape)

(792, 7)
(792,)


In [13]:
test_ds = ds[sp:]
len(test_ds)

198

In [14]:
test_X = []
test_y = []
for x, y in test_ds:
    test_X.append(x)
    test_y.append(y)

test_X = np.array(test_X)
print(test_X.shape)
test_y = np.array(test_y).squeeze()
print(test_y.shape)

(198, 7)
(198,)


## 模型定义

In [15]:
model = model = svm.SVC(gamma=0.001, C=100.)

# model = MLPClassifier(alpha=1e-5,
#                       activation='tanh')


## 训练

In [16]:
model.fit(train_X, train_y)

SVC(C=100.0, gamma=0.001)

## 预测

In [17]:
predict_y = model.predict(test_X)
predict_y

array([ 2000., 47200.,  3700.,  3800.,  2200.,  4400.,  2200.,  2400.,
        2500.,  7600.,  3100., 10200.,  1900.,  2100.,  1800.,  2000.,
        2000.,  2000.,  2000.,  1900.,  4700.,  1900.,  1800.,  2600.,
        2000.,  1800.,  1900., 18000.,  3700.,  1900.,  2000.,  1900.,
        1800.,  6300.,  2000.,  2000.,  2000.,  1800.,  1800.,  1800.,
        3800.,  2000.,  2000.,  1800.,  4700.,  4400.,  3700.,  2000.,
        2000.,  2000.,  2500.,  2000.,  2000.,  1800.,  5900., 14300.,
        2000.,  1800.,  5400.,  1800.,  2000.,  2000.,  6600.,  2600.,
        6700.,  1800.,  2300.,  1900.,  2000.,  4600.,  2000.,  2200.,
        2300.,  6300., 46100.,  2400.,  2000.,  1900.,  2700.,  2000.,
        2000.,  1800.,  2000.,  1800.,  5300.,  2500.,  2700.,  6300.,
       15000.,  2000.,  7200.,  3100.,  1800.,  1900.,  1800.,  6700.,
        1800.,  7600.,  2600.,  8100.,  6600.,  2000.,  1900.,  2000.,
        3000.,  1800.,  2500.,  3900.,  2000., 16600.,  1800.,  1800.,
      

In [18]:
test_y

array([ 3300., 13400.,  4200.,  2100.,  2300., 13100.,  2100.,  5300.,
       12100.,  2500.,  1900.,  6400.,  4400.,  1900.,  4600.,  4500.,
        2100.,  2900.,  3400., 12700.,  2200.,  7600.,  8200.,  2600.,
        5500.,  1900.,  3800.,  8500.,  4000.,  2900.,  1800.,  2200.,
        1900., 21700.,  4500.,  4100.,  2400.,  7000.,  2000.,  2000.,
        4900.,  3200.,  2900.,  2600.,  2300.,  2400.,  7900.,  2600.,
        2300.,  2100.,  2200.,  2600.,  2200.,  4600., 30300., 14300.,
        2500.,  2100.,  1800.,  2300.,  4000.,  3500., 11300.,  2600.,
        5300.,  8500.,  5800.,  2300.,  3700.,  2000.,  1800.,  8100.,
        2600., 23800.,  5200.,  6500.,  2600.,  5200.,  5000.,  2200.,
        2200.,  4500.,  3000.,  2600.,  5200.,  5800.,  5800., 16600.,
        2900.,  2100., 21400.,  3900.,  2100.,  3600., 15200.,  3600.,
        1800.,  5800.,  4500.,  7100., 11400.,  1900.,  3400.,  2300.,
       12000.,  2000.,  1900.,  4300.,  2000.,  7800.,  2300.,  7000.,
      

In [19]:
predict_y - test_y

array([ -1300.,  33800.,   -500.,   1700.,   -100.,  -8700.,    100.,
        -2900.,  -9600.,   5100.,   1200.,   3800.,  -2500.,    200.,
        -2800.,  -2500.,   -100.,   -900.,  -1400., -10800.,   2500.,
        -5700.,  -6400.,      0.,  -3500.,   -100.,  -1900.,   9500.,
         -300.,  -1000.,    200.,   -300.,   -100., -15400.,  -2500.,
        -2100.,   -400.,  -5200.,   -200.,   -200.,  -1100.,  -1200.,
         -900.,   -800.,   2400.,   2000.,  -4200.,   -600.,   -300.,
         -100.,    300.,   -600.,   -200.,  -2800., -24400.,      0.,
         -500.,   -300.,   3600.,   -500.,  -2000.,  -1500.,  -4700.,
            0.,   1400.,  -6700.,  -3500.,   -400.,  -1700.,   2600.,
          200.,  -5900.,   -300., -17500.,  40900.,  -4100.,   -600.,
        -3300.,  -2300.,   -200.,   -200.,  -2700.,  -1000.,   -800.,
          100.,  -3300.,  -3100., -10300.,  12100.,   -100., -14200.,
         -800.,   -300.,  -1700., -13400.,   3100.,      0.,   1800.,
        -1900.,   10

结果偏差较大，可能是数据质量不佳引起的。